In [1]:
from sklearn.model_selection import KFold, train_test_split
from xgboost import XGBRegressor
from scipy.stats import pearsonr
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import random
import warnings
import numpy as np 
import pandas as pd
import math
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Deep‑learning stack -------------------------------------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [17]:
class Config:
    TRAIN_PATH      = "train.parquet"
    TEST_PATH       = "test.parquet"
    SUBMISSION_PATH = "sample_submission.csv"
    
    FEATURES = [
        "X863", "X856", "X598", "X862", "X385", "X852", "X603", "X860", "X674",
        "X415", "X345", "X855", "X174", "X302", "X178", "X168", "X612", "bid_qty",
        "ask_qty", "buy_qty", "sell_qty", "volume", "X888", "X421", "X333",
        "X817", "X586", "X292"
    ]

    # Subset fed to MLP (slightly different cols)
    MLP_FEATURES = [
        "X863", "X856", "X598", "X862", "X385", "X603", "X860", "X674", "X415", "X345", "X855", "X174", "X302",
        "X178", "X168", "X612", "X333", "bid_qty", "ask_qty", "buy_qty", "sell_qty", "volume", 'log_volume',
        'bid_ask_interaction', 'bid_buy_interaction', 'bid_sell_interaction', 'ask_buy_interaction',
        'ask_sell_interaction']

    LABEL_COLUMN     = "label"
    N_FOLDS          = 3
    RANDOM_STATE     = 42
    OUTLIER_FRACTION = 0.001   # top 0.1 % residuals considered outliers
    OUTLIER_STRATEGIES = ["reduce", "remove", "double", "none"]

    lags = [1,3,5,10,20]
# XGBoost base parameters ---------------------------------------------------
XGB_PARAMS = {
    "tree_method": "hist",
    "device": "gpu" if torch.cuda.is_available() else "cpu",
    "colsample_bylevel": 0.4778,
    "colsample_bynode": 0.3628,
    "colsample_bytree": 0.7107,
    "gamma": 1.7095,
    "learning_rate": 0.02213,
    "max_depth": 20,
    "max_leaves": 12,
    "min_child_weight": 16,
    "n_estimators": 1667,
    "subsample": 0.06567,
    "reg_alpha": 39.3524,
    "reg_lambda": 75.4484,
    "verbosity": 0,
    "random_state": Config.RANDOM_STATE,
    "n_jobs": -1
}
LEARNERS = [
    {"name": "xgb", "Estimator": XGBRegressor, "params": XGB_PARAMS}
]

# ---------------------------------------------------------------------------
# Utility helpers (unchanged from modular version)
# ---------------------------------------------------------------------------

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def get_activation_function(name):
    if name is None:
        return None
    name = name.lower()
    if name == "relu":
        return nn.ReLU()
    if name == "tanh":
        return nn.Tanh()
    if name == "sigmoid":
        return nn.Sigmoid()
    raise ValueError(f"Unsupported activation: {name}")

In [5]:

class Checkpointer:
    """Simple best‑model saver"""
    def __init__(self, path="best_model.pt"):
        self.path         = path
        self.best_metric  = -np.inf

    def __call__(self, metric, model):
        if metric > self.best_metric:
            self.best_metric = metric
            torch.save(model.state_dict(), self.path)
            print(f"✅  New best saved ({metric:.4f}) → {self.path}")

    def load(self, model):
        model.load_state_dict(torch.load(self.path, map_location=device))
        return model


def get_dataloaders(X, y, hparams, shuffle=True):
    X_t = torch.tensor(X, dtype=torch.float32, device=device)
    if y is not None:
        y_t = torch.tensor(y, dtype=torch.float32, device=device).unsqueeze(1)
        ds  = TensorDataset(X_t, y_t)
    else:
        ds  = TensorDataset(X_t)
    return DataLoader(ds, batch_size=hparams["batch_size"], shuffle=shuffle,
                      generator=torch.Generator().manual_seed(hparams["seed"]))


In [19]:
# =========================
# Feature Engineering
# =========================
def add_features(df):
    df['bid_ask_interaction'] = df['bid_qty'] * df['ask_qty']
    df['bid_buy_interaction'] = df['bid_qty'] * df['buy_qty']
    df['bid_sell_interaction'] = df['bid_qty'] * df['sell_qty']
    df['ask_buy_interaction'] = df['ask_qty'] * df['buy_qty']
    df['ask_sell_interaction'] = df['ask_qty'] * df['sell_qty']

    df['volume_weighted_sell'] = df['sell_qty'] * df['volume']
    df['buy_sell_ratio'] = df['buy_qty'] / (df['sell_qty'])
    df['selling_pressure'] = df['sell_qty'] / (df['volume'])
    df['log_volume'] = np.log1p(df['volume'])

    df['effective_spread_proxy'] = np.abs(df['buy_qty'] - df['sell_qty']) / (df['volume'])
    df['bid_ask_imbalance'] = (df['bid_qty'] - df['ask_qty']) / (df['bid_qty'] + df['ask_qty'])
    df['order_flow_imbalance'] = (df['buy_qty'] - df['sell_qty']) / (df['buy_qty'] + df['sell_qty'])
    df['liquidity_ratio'] = (df['bid_qty'] + df['ask_qty']) / (df['volume'])

    FEATURES = [
        "X863", "X856", "X598", "X862", "X385", "X603", "X860", "X674", "X415", "X345", "X855", "X174", "X302",
        "X178", "X168", "X612", "X333", "bid_qty", "ask_qty", "buy_qty", "sell_qty", "volume", 'log_volume',
        'bid_ask_interaction', 'bid_buy_interaction', 'bid_sell_interaction', 'ask_buy_interaction',
        'ask_sell_interaction']
    
    lag_features = [f'{f}_lag_{l}' for f in FEATURES for l in Config.lags]
    lead_features = [f'{f}_lead_{l}' for f in FEATURES for l in Config.lags]

    for col in FEATURES:
        for lag in Config.lags:
            df[f'{col}_lag_{lag}'] = df[col].shift(lag)
            df[f'{col}_lead_{lag}'] = df[col].shift(-lag)

    df = df.fillna(df.mean())

    return df

def create_time_decay_weights(n: int, decay: float = 0.9) -> np.ndarray:
    positions = np.arange(n)
    normalized = positions / (n - 1) if n > 1 else positions
    weights = decay ** (1.0 - normalized)
    return weights * n / weights.sum()

def detect_outliers_and_adjust_weights(X, y, sample_weights, outlier_fraction=0.001, strategy="reduce"):
    """
    Detect outliers based on prediction residuals and adjust their weights.
    
    Strategies:
    - "reduce": Current approach - reduce weights to 0.2-0.8x
    - "remove": Set outlier weights to 0 (effectively removing them)
    - "double": Double the weights of outliers
    - "none": No adjustment
    """
    if strategy == "none":
        return sample_weights, np.zeros(len(y), dtype=bool)
    
    # Ensure we have at least some samples to detect outliers
    n_samples = len(y)
    if n_samples < 100:  # Not enough samples for meaningful outlier detection
        print(f"    Too few samples ({n_samples}) for outlier detection")
        return sample_weights, np.zeros(n_samples, dtype=bool)
    
    # Train a simple model to get residuals
    rf = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1)
    rf.fit(X, y, sample_weight=sample_weights)
    
    # Calculate residuals
    predictions = rf.predict(X)
    residuals = np.abs(y - predictions)
    
    # Find threshold for top outlier_fraction
    # Ensure we have at least 1 outlier
    n_outliers = max(1, int(len(residuals) * outlier_fraction))
    
    # Sort residuals and get threshold
    sorted_residuals = np.sort(residuals)
    threshold = sorted_residuals[-n_outliers] if n_outliers <= len(residuals) else sorted_residuals[-1]
    
    # Create outlier mask
    outlier_mask = residuals >= threshold
    
    # Ensure we have exactly n_outliers (handle ties at threshold)
    if np.sum(outlier_mask) > n_outliers:
        # If we have too many due to ties, randomly select to get exact number
        outlier_indices = np.where(outlier_mask)[0]
        np.random.seed(42)
        selected_indices = np.random.choice(outlier_indices, n_outliers, replace=False)
        outlier_mask = np.zeros(len(y), dtype=bool)
        outlier_mask[selected_indices] = True
    
    # Adjust weights based on strategy
    adjusted_weights = sample_weights.copy()
    
    if outlier_mask.any():
        if strategy == "reduce":
            # Original approach: reduce weights proportionally
            outlier_residuals = residuals[outlier_mask]
            min_outlier_res = outlier_residuals.min()
            max_outlier_res = outlier_residuals.max()
            
            if max_outlier_res > min_outlier_res:
                normalized_residuals = (outlier_residuals - min_outlier_res) / (max_outlier_res - min_outlier_res)
            else:
                normalized_residuals = np.ones_like(outlier_residuals)
            
            weight_factors = 0.8 - 0.6 * normalized_residuals
            adjusted_weights[outlier_mask] *= weight_factors
            
        elif strategy == "remove":
            # Set outlier weights to 0
            adjusted_weights[outlier_mask] = 0
            
        elif strategy == "double":
            # Double the weights of outliers
            adjusted_weights[outlier_mask] *= 2.0
        
        print(f"    Strategy '{strategy}': Adjusted {n_outliers} outliers ({outlier_fraction*100:.1f}% of data)")
    
    return adjusted_weights, outlier_mask

def load_data():
    # Load data with all features available
    all_features = list(set( Config.MLP_FEATURES))
    RAW_FEATURES = [
    "X863","X856","X598","X862","X385","X603","X860","X674","X415","X345",
    "X855","X174","X302","X178","X168","X612","X333",
    "bid_qty","ask_qty","buy_qty","sell_qty","volume"
    ]
    train_df = pd.read_parquet(Config.TRAIN_PATH, columns=RAW_FEATURES + [Config.LABEL_COLUMN])
    test_df = pd.read_parquet(Config.TEST_PATH, columns=RAW_FEATURES)
    submission_df = pd.read_csv(Config.SUBMISSION_PATH)
    print(f"Loaded data - Train: {train_df.shape}, Test: {test_df.shape}, Submission: {submission_df.shape}")

    # Add features
    train_df = add_features(train_df)
    test_df = add_features(test_df)

    return train_df.reset_index(drop=True), test_df.reset_index(drop=True), submission_df

def get_model_slices(n_samples: int):
    # Original 5 slices
    base_slices = [
        {"name": "full_data", "cutoff": 0, "is_oldest": False, "outlier_adjusted": False},
        {"name": "last_90pct", "cutoff": int(0.10 * n_samples), "is_oldest": False, "outlier_adjusted": False},
        {"name": "last_85pct", "cutoff": int(0.15 * n_samples), "is_oldest": False, "outlier_adjusted": False},
        {"name": "last_80pct", "cutoff": int(0.20 * n_samples), "is_oldest": False, "outlier_adjusted": False},
        {"name": "oldest_25pct", "cutoff": int(0.25 * n_samples), "is_oldest": True, "outlier_adjusted": False},
    ]
    
    # Duplicate slices with outlier adjustment
    outlier_adjusted_slices = []
    for slice_info in base_slices:
        adjusted_slice = slice_info.copy()
        adjusted_slice["name"] = f"{slice_info['name']}_outlier_adj"
        adjusted_slice["outlier_adjusted"] = True
        outlier_adjusted_slices.append(adjusted_slice)
    
    return base_slices + outlier_adjusted_slices

# =========================
# Outlier Analysis Functions
# =========================
def analyze_outliers(train_df):
    """Analyze outliers in the training data"""
    print("\n=== Outlier Analysis ===")
    
    X = train_df[Config.FEATURES].values
    y = train_df[Config.LABEL_COLUMN].values
    
    # Get base weights
    sample_weights = create_time_decay_weights(len(train_df))
    
    # Detect outliers
    _, outlier_mask = detect_outliers_and_adjust_weights(
        X, y, sample_weights, outlier_fraction=Config.OUTLIER_FRACTION, strategy="reduce"
    )
    
    # Analyze outlier characteristics
    outlier_indices = np.where(outlier_mask)[0]
    n_outliers = len(outlier_indices)
    
    print(f"\nTotal outliers detected: {n_outliers} ({n_outliers/len(train_df)*100:.2f}%)")
    
    if n_outliers > 0:
        # Statistical analysis
        outlier_labels = y[outlier_mask]
        normal_labels = y[~outlier_mask]
        
        print(f"\nLabel statistics:")
        print(f"  Normal samples - Mean: {normal_labels.mean():.4f}, Std: {normal_labels.std():.4f}")
        print(f"  Outlier samples - Mean: {outlier_labels.mean():.4f}, Std: {outlier_labels.std():.4f}")
        print(f"  Label range - Normal: [{normal_labels.min():.4f}, {normal_labels.max():.4f}]")
        print(f"  Label range - Outliers: [{outlier_labels.min():.4f}, {outlier_labels.max():.4f}]")
        
        # Feature analysis for outliers
        print(f"\nTop features with extreme values in outliers:")
        feature_names = Config.FEATURES[:20]  # Analyze first 20 features
        outlier_features = train_df.iloc[outlier_indices][feature_names]
        normal_features = train_df.iloc[~outlier_mask][feature_names]
        
        feature_diffs = []
        for feat in feature_names:
            outlier_mean = outlier_features[feat].mean()
            normal_mean = normal_features[feat].mean()
            if normal_mean != 0:
                rel_diff = abs(outlier_mean - normal_mean) / abs(normal_mean)
                feature_diffs.append((feat, rel_diff, outlier_mean, normal_mean))
        
        feature_diffs.sort(key=lambda x: x[1], reverse=True)
        for feat, diff, out_mean, norm_mean in feature_diffs[:10]:
            print(f"  {feat}: {diff*100:.1f}% difference (outlier: {out_mean:.4f}, normal: {norm_mean:.4f})")
    else:
        print("\nNo outliers detected with current threshold. Consider adjusting outlier_fraction.")
    
    return outlier_indices

# =========================
# XGBoost Training with Outlier Strategy Comparison
# =========================
def train_xgboost_with_outlier_comparison(train_df, test_df):
    """Train XGBoost with different outlier handling strategies and compare results"""
    n_samples = len(train_df)
    
    # Store results for each strategy
    strategy_results = {strategy: {"oof_scores": [], "slice_scores": {}} 
                       for strategy in Config.OUTLIER_STRATEGIES}
    
    # For final ensemble
    best_strategy = "reduce"  # Default to current approach
    best_score = -np.inf
    best_oof_preds = None
    best_test_preds = None
    
    for strategy in Config.OUTLIER_STRATEGIES:
        print(f"\n{'='*50}")
        print(f"Testing outlier strategy: {strategy.upper()}")
        print(f"{'='*50}")
        
        # Get model slices for this strategy
        model_slices = get_model_slices(n_samples)
        
        oof_preds = {
            learner["name"]: {s["name"]: np.zeros(n_samples) for s in model_slices}
            for learner in LEARNERS
        }
        test_preds = {
            learner["name"]: {s["name"]: np.zeros(len(test_df)) for s in model_slices}
            for learner in LEARNERS
        }
        
        full_weights = create_time_decay_weights(n_samples)
        kf = KFold(n_splits=Config.N_FOLDS, shuffle=False)
        
        for fold, (train_idx, valid_idx) in enumerate(kf.split(train_df), start=1):
            print(f"\n--- Fold {fold}/{Config.N_FOLDS} ---")
            X_valid = train_df.iloc[valid_idx][Config.FEATURES]
            y_valid = train_df.iloc[valid_idx][Config.LABEL_COLUMN]
            
            for s in model_slices:
                cutoff = s["cutoff"]
                slice_name = s["name"]
                is_oldest = s["is_oldest"]
                outlier_adjusted = s.get("outlier_adjusted", False)
                
                if is_oldest:
                    subset = train_df.iloc[:cutoff].reset_index(drop=True)
                    rel_idx = train_idx[train_idx < cutoff]
                    sw = np.ones(len(rel_idx))
                else:
                    subset = train_df.iloc[cutoff:].reset_index(drop=True)
                    rel_idx = train_idx[train_idx >= cutoff] - cutoff
                    sw = create_time_decay_weights(len(subset))[rel_idx] if cutoff > 0 else full_weights[train_idx]
                
                X_train = subset.iloc[rel_idx][Config.FEATURES]
                y_train = subset.iloc[rel_idx][Config.LABEL_COLUMN]
                
                # Apply outlier strategy if this is an outlier-adjusted slice
                if outlier_adjusted and len(X_train) > 100:
                    sw, _ = detect_outliers_and_adjust_weights(
                        X_train.values, 
                        y_train.values, 
                        sw, 
                        outlier_fraction=Config.OUTLIER_FRACTION,
                        strategy=strategy
                    )
                
                print(f"  Training slice: {slice_name}, samples: {len(X_train)}")
                
                for learner in LEARNERS:
                    model = learner["Estimator"](**learner["params"])
                    model.fit(X_train, y_train, sample_weight=sw, eval_set=[(X_valid, y_valid)], verbose=False)
                    
                    if is_oldest:
                        oof_preds[learner["name"]][slice_name][valid_idx] = model.predict(
                            train_df.iloc[valid_idx][Config.FEATURES]
                        )
                    else:
                        mask = valid_idx >= cutoff
                        if mask.any():
                            idxs = valid_idx[mask]
                            oof_preds[learner["name"]][slice_name][idxs] = model.predict(
                                train_df.iloc[idxs][Config.FEATURES]
                            )
                        if cutoff > 0 and (~mask).any():
                            base_slice_name = slice_name.replace("_outlier_adj", "")
                            if base_slice_name == slice_name:
                                fallback_slice = "full_data"
                            else:
                                fallback_slice = "full_data_outlier_adj"
                            oof_preds[learner["name"]][slice_name][valid_idx[~mask]] = oof_preds[learner["name"]][fallback_slice][
                                valid_idx[~mask]
                            ]
                    
                    test_preds[learner["name"]][slice_name] += model.predict(test_df[Config.FEATURES])
        
        # Normalize test predictions
        for learner_name in test_preds:
            for slice_name in test_preds[learner_name]:
                test_preds[learner_name][slice_name] /= Config.N_FOLDS
        
        # Evaluate this strategy
        learner_name = 'xgb'
        
        # Weights for ensemble
        weights = np.array([
            1.0,   # full_data
            1.0,   # last_90pct
            1.0,   # last_85pct
            1.0,   # last_80pct
            0,  # oldest_25pct
            0.9,   # full_data_outlier_adj
            0.9,   # last_90pct_outlier_adj
            0.9,   # last_85pct_outlier_adj
            0.9,   # last_80pct_outlier_adj
            0   # oldest_25pct_outlier_adj
        ])
        weights = weights / weights.sum()
        
        oof_weighted = pd.DataFrame(oof_preds[learner_name]).values @ weights
        test_weighted = pd.DataFrame(test_preds[learner_name]).values @ weights
        score_weighted = pearsonr(train_df[Config.LABEL_COLUMN], oof_weighted)[0]
        
        print(f"\n{strategy.upper()} Strategy - Weighted Ensemble Pearson: {score_weighted:.4f}")
        
        # Store individual slice scores
        slice_names = list(oof_preds[learner_name].keys())
        for i, slice_name in enumerate(slice_names):
            score = pearsonr(train_df[Config.LABEL_COLUMN], oof_preds[learner_name][slice_name])[0]
            strategy_results[strategy]["slice_scores"][slice_name] = score
            if "outlier_adj" in slice_name:
                print(f"  {slice_name}: {score:.4f} (weight: {weights[i]:.3f})")
        
        strategy_results[strategy]["oof_scores"].append(score_weighted)
        strategy_results[strategy]["ensemble_score"] = score_weighted
        strategy_results[strategy]["oof_preds"] = oof_weighted
        strategy_results[strategy]["test_preds"] = test_weighted
        
        # Track best strategy
        if score_weighted > best_score:
            best_score = score_weighted
            best_strategy = strategy
            best_oof_preds = oof_preds
            best_test_preds = test_preds
    
    # Print comparison summary
    print(f"\n{'='*50}")
    print("OUTLIER STRATEGY COMPARISON SUMMARY")
    print(f"{'='*50}")
    
    for strategy in Config.OUTLIER_STRATEGIES:
        score = strategy_results[strategy]["ensemble_score"]
        print(f"{strategy.upper()}: {score:.4f} {'← BEST' if strategy == best_strategy else ''}")
    
    # Analyze differences
    print(f"\nRelative performance vs 'reduce' strategy:")
    reduce_score = strategy_results["reduce"]["ensemble_score"]
    for strategy in Config.OUTLIER_STRATEGIES:
        if strategy != "reduce":
            score = strategy_results[strategy]["ensemble_score"]
            diff = (score - reduce_score) / reduce_score * 100
            print(f"  {strategy}: {diff:+.2f}%")
    
    return best_oof_preds, best_test_preds, model_slices, strategy_results, best_strategy

# =========================
# MLP Training (unchanged)
# =========================
def train_mlp(train_df, test_df):
    print("\n=== Training MLP Model ===")
    
    # Hyperparameters
    hparams = {
        "seed": 42,
        "num_epochs": 10,
        "batch_size": 1024 * 8 * 4,
        "learning_rate": 0.001,
        "weight_decay": 1e-3,
        "dropout_rate": 0.6,
        "layers": [len(Config.MLP_FEATURES), 256, 64, 1],
        "hidden_activation": None,
        "activation": "relu",
        "delta": 5,
        "noise_factor": 0.005
    }
    
    set_seed(hparams["seed"])
    
    # Prepare data for MLP
    X_train_full = train_df[Config.MLP_FEATURES].values
    y_train_full = train_df[Config.LABEL_COLUMN].values
    
    # Split for validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full, test_size=0.2, shuffle=False, random_state=42
    )
    
    # Scale data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(test_df[Config.MLP_FEATURES].values)
    
    # Create dataloaders
    train_loader = get_dataloaders(X_train, y_train, hparams, shuffle=True)
    val_loader = get_dataloaders(X_val, y_val, hparams, shuffle=False)
    test_loader = get_dataloaders(X_test, None, hparams, shuffle=False)
    
    # Initialize model
    model = MLP(
        layers=hparams["layers"],
        dropout_rate=hparams["dropout_rate"],
        activation=hparams["activation"],
        last_activation=hparams["hidden_activation"],
    ).to(device)
    
    criterion = nn.HuberLoss(delta=hparams["delta"], reduction='sum')
    optimizer = optim.Adam(model.parameters(), lr=hparams["learning_rate"], 
                          weight_decay=hparams["weight_decay"])
    
    checkpointer = Checkpointer(path="best_mlp_model.pt")
    
    # Training loop
    num_epochs = hparams["num_epochs"]
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Add noise for robustness
            inputs = inputs + torch.randn_like(inputs) * hparams["noise_factor"]
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            
        running_loss = running_loss / len(train_loader.dataset)
        print(f"Training Loss: {running_loss:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0.0
        preds = []
        trues = []
        with torch.no_grad():
            for inputs, targets in tqdm(val_loader, desc="Validation"):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
                preds.append(outputs.cpu().numpy())
                trues.append(targets.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        preds = np.concatenate(preds).flatten()
        trues = np.concatenate(trues).flatten()
        pearson_coef = pearsonr(preds, trues)[0]
        print(f"Validation Pearson Coef: {pearson_coef:.4f} | Loss: {val_loss:.4f}")

        checkpointer(pearson_coef, model)
    
    # Load best model and make predictions
    model = checkpointer.load(model)
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs in tqdm(test_loader, desc="Predicting"):
            inputs = inputs[0].to(device)
            outputs = model(inputs)
            predictions.append(outputs.cpu().numpy())

    predictions = np.concatenate(predictions).flatten()
    
    return predictions

# =========================
# Ensemble & Submission Functions
# =========================
def create_xgboost_submission(train_df, oof_preds, test_preds, submission_df, strategy="reduce"):
    learner_name = 'xgb'
    
    # Weights for 10 slices
    weights = np.array([
        1.0,   # full_data
        1.0,   # last_90pct
        1.0,   # last_85pct
        1.0,   # last_80pct
        0,  # oldest_25pct
        0.9,   # full_data_outlier_adj
        0.9,   # last_90pct_outlier_adj
        0.9,   # last_85pct_outlier_adj
        0.9,   # last_80pct_outlier_adj
        0    # oldest_25pct_outlier_adj
    ])
    
    # Normalize weights
    weights = weights / weights.sum()

    oof_weighted = pd.DataFrame(oof_preds[learner_name]).values @ weights
    test_weighted = pd.DataFrame(test_preds[learner_name]).values @ weights
    score_weighted = pearsonr(train_df[Config.LABEL_COLUMN], oof_weighted)[0]
    print(f"\n{learner_name.upper()} Weighted Ensemble Pearson: {score_weighted:.4f}")

    # Print individual slice scores and weights for analysis
    print("\nIndividual slice OOF scores and weights:")
    slice_names = list(oof_preds[learner_name].keys())
    for i, slice_name in enumerate(slice_names):
        score = pearsonr(train_df[Config.LABEL_COLUMN], oof_preds[learner_name][slice_name])[0]
        print(f"  {slice_name}: {score:.4f} (weight: {weights[i]:.3f})")

    # Save XGBoost submission
    xgb_submission = submission_df.copy()
    xgb_submission["prediction"] = test_weighted
    xgb_submission.to_csv(f"submission_xgboost_{strategy}.csv", index=False)
    print(f"\nSaved: submission_xgboost_{strategy}.csv")
    
    return test_weighted, oof_weighted

def create_ensemble_submission(xgb_predictions, mlp_predictions, submission_df, 
                             xgb_weight=0.9, mlp_weight=0.1, suffix=""):
    # Ensemble predictions
    ensemble_predictions = (xgb_weight * xgb_predictions + 
                          mlp_weight * mlp_predictions)
    
    # Save ensemble submission
    ensemble_submission = submission_df.copy()
    ensemble_submission["prediction"] = ensemble_predictions
    filename = f"submission_ensemble{suffix}.csv"
    ensemble_submission.to_csv(filename, index=False)
    print(f"\nSaved: {filename} (XGBoost: {xgb_weight*100}%, MLP: {mlp_weight*100}%)")
    
    return ensemble_predictions


In [1]:
train_df, test_df, submission_df = load_data()

In [8]:
# Analyze outliers
outlier_indices = analyze_outliers(train_df)


=== Outlier Analysis ===
    Strategy 'reduce': Adjusted 525 outliers (0.1% of data)

Total outliers detected: 525 (0.10%)

Label statistics:
  Normal samples - Mean: 0.0377, Std: 0.9746
  Outlier samples - Mean: -1.5579, Std: 8.2826
  Label range - Normal: [-24.4166, 20.7403]
  Label range - Outliers: [-16.2274, 13.1532]

Top features with extreme values in outliers:
  X345: 830.6% difference (outlier: -0.1733, normal: 0.0237)
  X598: 626.4% difference (outlier: -0.1587, normal: 0.0301)
  X863: 409.8% difference (outlier: 0.0292, normal: -0.0094)
  buy_qty: 316.4% difference (outlier: 546.7665, normal: 131.3119)
  X385: 128.5% difference (outlier: -0.0201, normal: 0.0704)
  X856: 109.8% difference (outlier: -0.0271, normal: -0.0129)
  X178: 84.9% difference (outlier: 0.0050, normal: 0.0332)
  X168: 83.7% difference (outlier: 0.0241, normal: 0.1475)
  X174: 80.9% difference (outlier: 0.0281, normal: 0.1474)
  X603: 79.2% difference (outlier: 0.2810, normal: 0.1569)


In [9]:
# Train XGBoost with outlier comparison
print("\n=== Training XGBoost Models with Outlier Strategy Comparison ===")
best_oof_preds, best_test_preds, model_slices, strategy_results, best_strategy = \
    train_xgboost_with_outlier_comparison(train_df, test_df)

# Create XGBoost submission with best strategy
xgb_predictions, oof_weighted = create_xgboost_submission(
    train_df, best_oof_preds, best_test_preds, submission_df, strategy=best_strategy
)




=== Training XGBoost Models with Outlier Strategy Comparison ===

Testing outlier strategy: REDUCE

--- Fold 1/3 ---
  Training slice: full_data, samples: 350591
  Training slice: last_90pct, samples: 350591
  Training slice: last_85pct, samples: 350591
  Training slice: last_80pct, samples: 350591
  Training slice: oldest_25pct, samples: 0
    Strategy 'reduce': Adjusted 350 outliers (0.1% of data)
  Training slice: full_data_outlier_adj, samples: 350591
    Strategy 'reduce': Adjusted 350 outliers (0.1% of data)
  Training slice: last_90pct_outlier_adj, samples: 350591
    Strategy 'reduce': Adjusted 350 outliers (0.1% of data)
  Training slice: last_85pct_outlier_adj, samples: 350591
    Strategy 'reduce': Adjusted 350 outliers (0.1% of data)
  Training slice: last_80pct_outlier_adj, samples: 350591
  Training slice: oldest_25pct_outlier_adj, samples: 0

--- Fold 2/3 ---
  Training slice: full_data, samples: 350591
  Training slice: last_90pct, samples: 298003
  Training slice: las

In [61]:
def train_mlp(train_df, test_df):
    print("\n=== Training MLP Model ===")
    
    # Hyperparameters
    hparams = {
        "seed": 42,
        "num_epochs": 10,
        "batch_size": 1024 * 6,
        "learning_rate": 6e-4,
        "weight_decay": 6e-3,
        "dropout_rate": 0.5,
        "layers": [len(Config.MLP_FEATURES), 128, 32, 1],
        "hidden_activation": None,
        "activation": "relu",
        "delta": 5,
        "noise_factor": 0.005
    }
    
    set_seed(hparams["seed"])
    
    # Prepare data for MLP
    X_train_full = train_df[Config.MLP_FEATURES].values
    y_train_full = train_df[Config.LABEL_COLUMN].values
    
    # Split for validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full, test_size=0.2, shuffle=False, random_state=42
    )
    
    # Scale data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(test_df[Config.MLP_FEATURES].values)
    
    # Create dataloaders
    train_loader = get_dataloaders(X_train, y_train, hparams, shuffle=True)
    val_loader = get_dataloaders(X_val, y_val, hparams, shuffle=False)
    test_loader = get_dataloaders(X_test, None, hparams, shuffle=False)
    
    # Initialize model
    model = MLP(
        layers=hparams["layers"],
        dropout_rate=hparams["dropout_rate"],
        activation=hparams["activation"],
        last_activation=hparams["hidden_activation"],
    ).to(device)
    
    criterion = nn.HuberLoss(delta=hparams["delta"], reduction='sum')
    optimizer = optim.Adam(model.parameters(), lr=hparams["learning_rate"], 
                          weight_decay=hparams["weight_decay"])
    
    checkpointer = Checkpointer(path="best_mlp_model.pt")
    
    # Training loop
    num_epochs = hparams["num_epochs"]
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Add noise for robustness
            inputs = inputs + torch.randn_like(inputs) * hparams["noise_factor"]
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            
        running_loss = running_loss / len(train_loader.dataset)
        print(f"Training Loss: {running_loss:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0.0
        preds = []
        trues = []
        with torch.no_grad():
            for inputs, targets in tqdm(val_loader, desc="Validation"):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
                preds.append(outputs.cpu().numpy())
                trues.append(targets.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        preds = np.concatenate(preds).flatten()
        trues = np.concatenate(trues).flatten()
        pearson_coef = pearsonr(preds, trues)[0]
        print(f"Validation Pearson Coef: {pearson_coef:.4f} | Loss: {val_loss:.4f}")

        checkpointer(pearson_coef, model)
    
    # Load best model and make predictions
    model = checkpointer.load(model)
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs in tqdm(test_loader, desc="Predicting"):
            inputs = inputs[0].to(device)
            outputs = model(inputs)
            predictions.append(outputs.cpu().numpy())

    predictions = np.concatenate(predictions).flatten()
    
    return predictions


class MLP(nn.Module):
    def __init__(self, layers, d_model=128, nhead=8, num_layers=2,
                 dropout_rate=0.5, activation="relu", last_activation=None):
        super().__init__()
        self.d_model = d_model
        self.token_proj = nn.Linear(1, d_model)

        # Transformer
        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=d_model*4,
            dropout=dropout_rate, activation="gelu", batch_first=False
        )
        self.transformer = nn.TransformerEncoder(enc_layer, num_layers=num_layers)

        # MLP 头：首层用 d_model 而不是原 feature 数
        self.linears = nn.ModuleList(
            [nn.Linear(d_model if i == 0 else layers[i], layers[i+1])
             for i in range(len(layers)-1)]
        )
        self.act, self.last_act = get_activation_function(activation), get_activation_function(last_activation)
        self.dropout = nn.Dropout(dropout_rate)

    def _get_pos_emb(self, seq_len, device):
        """按当前长度生成正弦位置编码 (seq_len, 1, d_model)"""
        pe = torch.zeros(seq_len, self.d_model, device=device)
        position = torch.arange(seq_len, dtype=torch.float, device=device).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, self.d_model, 2, device=device).float()
                             * (-math.log(10000.0) / self.d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(1)            # (seq_len, 1, d_model)

    def forward(self, x):                 # x : (batch, feature_dim)
        seq_len = x.size(1)
        x_tok   = x.transpose(0, 1).unsqueeze(-1)        # (seq_len, batch, 1)
        x_tok   = self.token_proj(x_tok)                 # (seq_len, batch, d_model)

        pos_emb = self._get_pos_emb(seq_len, x_tok.device)
        x_tok   = x_tok + pos_emb                        # 加位置
        x_tok   = self.transformer(x_tok)                # (seq_len, batch, d_model)
        h       = x_tok.mean(dim=0)                      # (batch, d_model)

        # MLP 头
        for layer in self.linears[:-1]:
            h = self.dropout(self.act(layer(h)))
        h = self.linears[-1](h)
        if self.last_act:
            h = self.last_act(h)
        return h

In [62]:
# Train MLP model
mlp_predictions = train_mlp(train_df, test_df)
# mlp_oof, mlp_predictions = train_mlp_kfold(train_df, test_df)

# Save MLP submission
mlp_submission = submission_df.copy()
mlp_submission["prediction"] = mlp_predictions
mlp_submission.to_csv("submission_mlp.csv", index=False)
print("\nSaved: submission_mlp.csv")


=== Training MLP Model ===


/venv/main/lib/python3.12/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
Epoch 1/10: 100%|██████████| 69/69 [00:11<00:00,  5.99it/s]


Training Loss: 3000.5498


Validation: 100%|██████████| 18/18 [00:01<00:00, 14.07it/s]


Validation Pearson Coef: 0.0251 | Loss: 3234.8294
✅  New best saved (0.0251) → best_mlp_model.pt


...

Training Loss: 2807.0844


Validation: 100%|██████████| 18/18 [00:01<00:00, 13.99it/s]


Validation Pearson Coef: 0.0593 | Loss: 6012.3112


Predicting: 100%|██████████| 88/88 [00:04<00:00, 18.08it/s]



Saved: submission_mlp.csv


In [ ]:
# Create ensemble submission
ensemble_predictions = create_ensemble_submission(
    xgb_predictions, mlp_predictions, submission_df,
    xgb_weight=0.9, mlp_weight=0.1, suffix=f"_{best_strategy}"
)
